In [1]:
DICT = "ABCDEFGHIJKLMNOPQRSTUVWXYZ" + "abcdefghijklmnopqrstuvwxyz" + "0123456789" + "@#"


def decode_text(text: str) -> str:
    decoded = ""
    for char in list(text):
        index = DICT.index(char)
        decoded += format(index, "b").zfill(6)

    return decoded

In [2]:
expected = "000000" + "000001" + "000010"
actual = decode_text("ABC")
assert actual == expected

In [3]:
expected = "000111" + "111110" + "111111"
actual = decode_text("H@#")
assert actual == expected

In [4]:
def read_line(file_name: str) -> str:
    with open(file_name, "r") as f:
        return f.readline().strip()

In [5]:
def question1():
    text = read_line("data/data1_test.txt")
    decoded = decode_text(text)
    print(f"{decoded[310:320]=}")


question1()

decoded[310:320]='0010100011'


In [6]:
def unpack(bits: str) -> str:
    decoded = ""
    i = 0
    while True:
        if len(bits) <= i:
            break

        byte = bits[i : i + 8]
        if byte != "00000000":
            decoded += byte
            i += 8
        else:
            p_byte, d_byte = bits[i + 8 : i + 16], bits[i + 16 : i + 24]
            print(f"{p_byte=}, {d_byte=}")
            if d_byte == "00000000":
                decoded += "00000000"
            else:
                p, d = int(p_byte, 2), int(d_byte, 2)
                prev = ""
                if p - d + 1 == 1:
                    prev = decoded[-p * 8 :]
                else:
                    prev = decoded[-p * 8 : -(p - d) * 8]
                print(f"{p=}, {d=}, {prev=}")
                decoded += prev
            i += 24

    return decoded

In [7]:
def h2b(hex: str) -> str:
    digits = len(hex) * 4
    return format(int(hex, 16), "b").lstrip("0b").zfill(digits)

In [8]:
def b2h(bits: str) -> str:
    if bits == "":
        return bits
    return {format(int(bits, 2), "x")}

In [9]:
assert h2b("00") == "00000000"
assert h2b("FF") == "11111111"
assert h2b("EEEE") == "1110111011101110"

In [10]:
expected = h2b("112233")
actual = unpack(h2b("112233"))
assert actual == expected

In [11]:
expected = h2b("41424344454647424344454648")
actual = unpack(h2b("4142434445464700060548"))
print(f"{b2h(actual)=}, {b2h(expected)=}")
assert actual == expected

p_byte='00000110', d_byte='00000101'
p=6, d=5, prev='0100001001000011010001000100010101000110'
b2h(actual)={'41424344454647424344454648'}, b2h(expected)={'41424344454647424344454648'}


In [12]:
from collections import defaultdict


def wrap_encrypt_dp(bits: str) -> str:
    # NOTE: dpオブジェクトをセルのスコープに置くと、前回のループの状態を引き継いでしまう!!!
    dp = defaultdict(lambda: None)
    dp[0] = ""

    def encrypt_dp(bits: str) -> str:
        if dp[len(bits)] is not None:
            return dp[len(bits)]

        if len(bits) < 64:
            dp[len(bits)] = bits
            return bits

        best_encryption = None
        for 複製探索中の右端 in range(len(bits), 63, -8):
            for 複製探索中のbit数 in range(32, min(複製探索中の右端, 256 * 8), 8):
                # 非常に細かいことを言うと、通常は圧縮してお得になるのは4Byteからだが、0x00が含まれる場合は2Byteからお得になる
                # しかし時間制限がある中で流石にそこまでケアできない
                encrypted = bits[複製探索中の右端:].replace(
                    "00000000", "00000000" + "00000000" + "00000000"
                )
                複製探索中の左端 = 複製探索中の右端 - 複製探索中のbit数
                try:
                    start_index = bits[0:複製探索中の左端].index(
                        bits[複製探索中の左端:複製探索中の右端]
                    )
                    print(
                        f"{b2h(bits[0:複製探索中の左端])}, {b2h(bits[複製探索中の左端:複製探索中の右端])}, {b2h(bits[複製探索中の右端:])}, Found"
                    )
                    返るbit数 = 複製探索中の左端 - start_index
                    p_bit = format(返るbit数 // 8, "b").lstrip("0b").zfill(8)
                    d_bit = format(複製探索中のbit数 // 8, "b").lstrip("0b").zfill(8)
                    encrypted = (
                        encrypt_dp(bits[0:複製探索中の左端])
                        + "00000000"
                        + p_bit
                        + d_bit
                        + encrypted
                    )
                    print(
                        f"{b2h(bits[0:複製探索中の左端])}, {b2h(bits[複製探索中の左端:複製探索中の右端])}, {b2h(bits[複製探索中の右端:])}, {b2h(encrypted)=}"
                    )
                except ValueError:
                    print(
                        f"{b2h(bits[0:複製探索中の左端])}, {b2h(bits[複製探索中の左端:複製探索中の右端])}, {b2h(bits[複製探索中の右端:])}, NotFound"
                    )
                    encrypted = (
                        bits[0:複製探索中の右端].replace(
                            "00000000", "00000000" + "00000000" + "00000000"
                        )
                        + encrypted
                    )
                except TypeError as e:
                    print(e)
                finally:
                    if best_encryption is None or len(encrypted) < len(best_encryption):
                        best_encryption = encrypted

        dp[len(bits)] = best_encryption
        return best_encryption

    return encrypt_dp(bits)

In [13]:
expected = h2b("00")
actual = wrap_encrypt_dp(h2b("00"))
assert actual == expected

In [14]:
expected = h2b("11223344000404")
actual = wrap_encrypt_dp(h2b("1122334411223344"))
print(f"{b2h(actual)=}, {b2h(expected)=}")
assert actual == expected

{'11223344'}, {'11223344'}, , Found
{'11223344'}, {'11223344'}, , b2h(encrypted)={'11223344000404'}
{'112233'}, {'4411223344'}, , NotFound
{'1122'}, {'334411223344'}, , NotFound
{'11'}, {'22334411223344'}, , NotFound
b2h(actual)={'11223344000404'}, b2h(expected)={'11223344000404'}


In [16]:
expected = h2b("4142434445464700060548")
actual = wrap_encrypt_dp(h2b("41424344454647424344454648"))
assert actual == expected

{'414243444546474243'}, {'44454648'}, , NotFound
{'4142434445464742'}, {'4344454648'}, , NotFound
{'41424344454647'}, {'424344454648'}, , NotFound
{'414243444546'}, {'47424344454648'}, , NotFound
{'4142434445'}, {'4647424344454648'}, , NotFound
{'41424344'}, {'454647424344454648'}, , NotFound
{'414243'}, {'44454647424344454648'}, , NotFound
{'4142'}, {'4344454647424344454648'}, , NotFound
{'41'}, {'424344454647424344454648'}, , NotFound
{'4142434445464742'}, {'43444546'}, {'48'}, Found
{'41424344'}, {'45464742'}, , NotFound
{'414243'}, {'4445464742'}, , NotFound
{'4142'}, {'434445464742'}, , NotFound
{'41'}, {'42434445464742'}, , NotFound
{'4142434445464742'}, {'43444546'}, {'48'}, b2h(encrypted)={'414243444546474200060448'}
{'41424344454647'}, {'4243444546'}, {'48'}, Found
{'41424344454647'}, {'4243444546'}, {'48'}, b2h(encrypted)={'4142434445464700060548'}
{'414243444546'}, {'474243444546'}, {'48'}, NotFound
{'4142434445'}, {'46474243444546'}, {'48'}, NotFound
{'41424344'}, {'4546474